In [1]:
# TO RUN IT: CHANGE MY PATH
%matplotlib inline
%load_ext autoreload
%autoreload 2

# change path if necessary
import sys
my_path = r'D:\Documents\etudes\epfl\MA1\cours\MachineLearning\Project1'
sys.path.insert(0,my_path + r'\code\COMMON')

# imports
import numpy as np 
import matplotlib.pyplot as plt

from implementations import *
from labels import idx_2labels
from costs import *
from optimize_hyperparams import *
from cross_validation import *
from step_wise import *

# Build dataset

In [2]:
from proj1_helpers import load_csv_data 

# load raw data
y_raw, input_data_raw, ids = load_csv_data(my_path + r'\data\train.csv', sub_sample=False)

In [3]:
from outliers import handle_outliers

# handle outliers
X0, y = handle_outliers(input_data_raw, y_raw, -999, 'median')

-999 are replaced by the median value of the feature


In [13]:
from standard import standardize

# standardize
X0, mean_X0, std_X0 = standardize(X0)

# set y in {0,1} instead of {-1,1}
y[np.where(y==-1)]=0

In [48]:
# get feature names 
all_features_raw = list(np.genfromtxt(my_path + r'/data/train.csv', delimiter=",", dtype=str, max_rows = 1)[2:])

In [50]:
# adding power transformations
degree = 3

# build polynomial basis function
phi = build_poly(X0, degree)

# standardize the degree > 1
phi[:, 1:],_ ,_ = standardize(phi[:, 1:])

# create feature set
all_candidates = phi

# adding logarithm names
all_features = ['offset']
for i in range(len(all_features_raw)):
    for d in range(degree):
        all_features.append(all_features_raw[i] + '_power_' + str(d+1))

In [51]:
# adding logarithmic transformation of masses
all_candidates = np.concatenate((all_candidates, np.log(1+np.abs(X0[:,0])).reshape(len(X0),1)),axis =1)
all_candidates = np.concatenate((all_candidates, np.log(1+np.abs(X0[:,1])).reshape(len(X0),1)),axis =1)
all_candidates = np.concatenate((all_candidates, np.log(1+np.abs(X0[:,2])).reshape(len(X0),1)),axis =1)
all_candidates = np.concatenate((all_candidates, np.log(1+np.abs(X0[:,5])).reshape(len(X0),1)),axis =1)

# adding logarithm names
all_features.append('log(DER_mass_MMC)')
all_features.append('log(DER_mass_transverse_met_lep)')
all_features.append('log(DER_mass_vis)')
all_features.append('log(DER_mass_jet_jet)')

In [53]:
# list of feature names and indices
features = []
for i in range(len(all_features)):
    features.append((i,all_features[i]))

print(all_candidates.shape)
print(len(features))
print(features)

(250000, 95)
95
[(0, 'offset'), (1, 'DER_mass_MMC_power_1'), (2, 'DER_mass_MMC_power_2'), (3, 'DER_mass_MMC_power_3'), (4, 'DER_mass_transverse_met_lep_power_1'), (5, 'DER_mass_transverse_met_lep_power_2'), (6, 'DER_mass_transverse_met_lep_power_3'), (7, 'DER_mass_vis_power_1'), (8, 'DER_mass_vis_power_2'), (9, 'DER_mass_vis_power_3'), (10, 'DER_pt_h_power_1'), (11, 'DER_pt_h_power_2'), (12, 'DER_pt_h_power_3'), (13, 'DER_deltaeta_jet_jet_power_1'), (14, 'DER_deltaeta_jet_jet_power_2'), (15, 'DER_deltaeta_jet_jet_power_3'), (16, 'DER_mass_jet_jet_power_1'), (17, 'DER_mass_jet_jet_power_2'), (18, 'DER_mass_jet_jet_power_3'), (19, 'DER_prodeta_jet_jet_power_1'), (20, 'DER_prodeta_jet_jet_power_2'), (21, 'DER_prodeta_jet_jet_power_3'), (22, 'DER_deltar_tau_lep_power_1'), (23, 'DER_deltar_tau_lep_power_2'), (24, 'DER_deltar_tau_lep_power_3'), (25, 'DER_pt_tot_power_1'), (26, 'DER_pt_tot_power_2'), (27, 'DER_pt_tot_power_3'), (28, 'DER_sum_pt_power_1'), (29, 'DER_sum_pt_power_2'), (30, 'DER

# Logistic Regression (no penalization)

In [56]:
# model parameters
model = dict()
model['method'] = 'lr' # 'lrr' if regularized with a penalization 'lambda_' otherwise 'lr' 
model['loss'] = 'loglikelihood'

# model hyperparameter
model['gamma'] = 1e-5

# other
model['method_minimization'] = 'gd' # 'gd' (gradient_descent) or 'newton' (newton descent leads to non invertible matrice "S" which is too big)
model['threshold'] = 1
model['max_iters'] = 10000
model['debug_mode'] = 0

# R2 type
R2_method = 'McFadden' # 'loss', 'Tjur' or 'McFadden' 

# estimate R2 error through cross validation (1 or 0)
cv = 1
model['k_fold'] = 10

In [57]:
# step-wise
best_R2adj, idx_features = stepwise(model, R2_method, all_candidates, features, y, cv)

KeyboardInterrupt: 

In [ ]:
# display selected features
results_r2_stepwise(best_R2adj[:len(best_R2adj)-1], idx_features)